In [1]:
import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
from cupyx.profiler import benchmark
from matplotlib import pyplot as plt
import os
import json
import pickle


from qwak_cupy.qwak import QWAK as CQWAK
from qwak.qwak import QWAK as QWAK


In [2]:
def load_list_from_file(file_path):
    with open(file_path, 'r') as file:
        data_str = file.read()
    data = [json.loads(line) for line in data_str.splitlines()]
    return data

def write_list_to_file(file_path, data):
    data_str = [str(item) for item in data]  # Convert float values to strings
    with open(file_path, 'w') as file:
        file.write('\n'.join(data_str))
t = 50

In [3]:
def runTimedQWAK(n,t,graph):
    start_time = time.time()
    initNodes = [n//2]
    qw = QWAK(graph)
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    return qw, qwak_time

def runTimedQWAK_cupy(n,t,graph):
    start_time = time.time()
    initNodes = [n//2]
    qw = CQWAK(graph)
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    return qw, qwak_time

def runMultipleSimpleQWAK(nList,t,samples,graphList_list):
    qwList = []
    tList = []
    qwak_time = 0
    qwak_time_average = 0
    print(f"Running NP QWAK {len(nList)} walks up to n = {nList[-1]}")
    for n,graphList in zip(nList,graphList_list):
        for sample,graph in zip(range(1,samples+1),graphList):
            print(f"----> Calculating NP QWAK for n = {n} \t Sample #{sample}",end='\r')
            qw, qwak_time = runTimedQWAK(n,t,graph)
            qwak_time_average += qwak_time 
        qwak_time_average = qwak_time_average / samples
        qwList.append(qw)
        tList.append(qwak_time_average)
        qwak_time_average = 0
    return tList, qwList

def runMultipleSimpleQWAK_cupy(nList,t,samples,graphList_list):
    qwList = []
    tList = []
    qwak_time = 0
    qwak_time_average = 0
    print(f"Running CuPy QWAK {len(nList)} walks up to n = {nList[-1]}")
    for n,graphList in zip(nList,graphList_list):
        # print(f"----> Calculating NP QWAK for n = {n}",end='\r')
        for sample,graph in zip(range(1,samples+1),graphList):
            print(f"----> Calculating CuPy QWAK for n = {n} \t Sample #{sample}",end='\r')
            qw, qwak_time = runTimedQWAK_cupy(n,t,graph)
            qwak_time_average += qwak_time 
        qwak_time_average = qwak_time_average / samples
        qwList.append(qw)
        tList.append(qwak_time_average)
        qwak_time_average = 0
    return tList, qwList

def runTimedQWAK2(n,t):
    start_time = time.time()
    initNodes = [n//2]
    qw = QWAK(nx.cycle_graph(n))
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    return qwak_time

def runTimedQWAK_cupy2(n,t):
    start_time = time.time()
    initNodes = [n//2]
    qw = CQWAK(nx.cycle_graph(n))
    qw.runWalk(t, initNodes)
    end_time = time.time()
    qwak_time = end_time - start_time
    return qwak_time

def runMultipleSimpleQWAK_cupy2(nList,t,samples):
    tList = []
    qwak_time = 0
    qwak_time_average = 0
    print(f"Running CuPy QWAK {len(nList)} walks up to n = {nList[-1]}")
    for n in nList:
        # print(f"----> Calculating NP QWAK for n = {n}",end='\r')
        for sample in range(1,samples+1):
            print(f"----> Calculating CuPy QWAK for n = {n} \t Sample #{sample}",end='\r')
            qwak_time = runTimedQWAK_cupy2(n,t)
            qwak_time_average += qwak_time 
        qwak_time_average = qwak_time_average / samples
        # qwList.append(qw)
        tList.append(qwak_time_average)
        qwak_time_average = 0
    return tList#, qwList

def runMultipleSimpleQWAK2(nList,t,samples):
    #qwList = []
    tList = []
    qwak_time = 0
    qwak_time_average = 0
    print(f"Running NP QWAK {len(nList)} walks up to n = {nList[-1]}")
    for n in nList:
        for sample in range(1,samples+1):
            print(f"----> Calculating NP QWAK for n = {n} \t Sample #{sample}",end='\r')
            qwak_time = runTimedQWAK2(n,t)
            qwak_time_average += qwak_time 
        qwak_time_average = qwak_time_average / samples
#        qwList.append(qw)
        tList.append(qwak_time_average)
        qwak_time_average = 0
    return tList#, qwList


def create_or_load_graph_list(filepath, n_values, pVal, samples):
    """
    Checks if a file exists at the specified path and either loads a list of lists of graphs
    or creates and saves it if the file does not exist.
    
    Parameters:
        filepath (str): The full file path to check and save/load the serialized list of lists.
        n_values (list of int): A list of n values (graph sizes).
        pVal (float): The probability for edge creation in the Erdos-Renyi model.
        samples (int): The number of graphs to generate for each n value.
    
    Returns:
        list: A list of lists containing the generated or loaded graphs.
    """
    # Ensure the directory for the file exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    
    # Check if the file already exists
    if os.path.exists(filepath):
        print(f"File {filepath} found. Loading graphs...")
        with open(filepath, 'rb') as f:
            graph_list_of_lists = pickle.load(f)
        return graph_list_of_lists
    
    print(f"File {filepath} not found. Creating graphs...")
    # Create a list of lists to hold the graphs
    graph_list_of_lists = []
    
    for n in n_values:
        # Create a list to hold graphs of size 'n'
        graph_list = []
        for _ in range(samples):
            # Generate the graph and append it to the graph_list
            graph = nx.erdos_renyi_graph(n, pVal)
            graph_list.append(graph)
        # Append the graph_list to the graph_list_of_lists
        graph_list_of_lists.append(graph_list)
    
    # Save the list of lists to a file
    with open(filepath, 'wb') as f:
        pickle.dump(graph_list_of_lists, f)
    print(f"Graph list of lists saved to {filepath}")
    
    return graph_list_of_lists



In [4]:
nMin = 3
nMax = 301
nList = list(range(nMin,nMax,1))
pVal = 0.8
samples = 2
t = 10

qwak_times_filename = f'LINUX-simpleQWAKTime_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
qwak_times_filename_cupy = f'LINUX-simpleQWAKTime_CuPy_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'
graphs_filename = f'LINUX-Graph-simpleQWAKTime_N{nMin}-{nMax-1}_P{pVal}_T{t}_S{samples}.txt'

qwak_times_file = f'Datasets/Benchmark-SimpleQWAK_ER/' + qwak_times_filename
qwak_times_file_cupy = f'Datasets/Benchmark-SimpleQWAK_ER/' + qwak_times_filename_cupy

graph_file = f'Datasets/Benchmark-SimpleQWAK_ER/Graphs/' + graphs_filename

graph_list = create_or_load_graph_list(graph_file, nList, pVal, samples)

if os.path.exists(qwak_times_file):
     qwak_times = load_list_from_file(qwak_times_file)
     print('File Exists!')
else:
     qwak_times,qw = runMultipleSimpleQWAK(nList,t,samples,graph_list)
     write_list_to_file(qwak_times_file,qwak_times)

if os.path.exists(qwak_times_file_cupy):
    qwak_times_cupy = load_list_from_file(qwak_times_file_cupy)
    print('File Exists!')
else:
    qwak_times_cupy,qw_cupy = runMultipleSimpleQWAK_cupy(nList,t,samples,graph_list)
    write_list_to_file(qwak_times_file_cupy,qwak_times_cupy)


File Datasets/Benchmark-SimpleQWAK_ER/Graphs/LINUX-Graph-simpleQWAKTime_N3-300_P0.8_T10_S2.txt not found. Creating graphs...


KeyboardInterrupt: 

In [ ]:
import os
import networkx as nx
import pickle
import numpy as np

def create_or_load_adjacency_matrices(base_dir, n_values, pVal, samples):
    """
    Checks if adjacency matrix files exist for each size `n` and each sample. If not, generates and saves them.

    Parameters:
        base_dir (str): The base directory to save/load the adjacency matrices.
        n_values (list of int): A list of n values (graph sizes).
        pVal (float): The probability for edge creation in the Erdos-Renyi model.
        samples (int): The number of adjacency matrices to generate for each n value.

    Returns:
        dict: A dictionary where keys are `n` values and values are lists of adjacency matrices.
    """
    adjacency_matrix_dict = {}

    for n in n_values:
        n_dir = os.path.join(base_dir, f"N{n}")  # Directory for this n value
        os.makedirs(n_dir, exist_ok=True)

        adjacency_matrix_list = []
        for sample in range(samples):
            sample_file = os.path.join(n_dir, f"AdjMatrix_N{n}_Sample{sample}.pkl")

            if os.path.exists(sample_file):
                # Load existing adjacency matrix
                with open(sample_file, 'rb') as f:
                    adjacency_matrix = pickle.load(f)
            else:
                # Generate and save a new adjacency matrix
                graph = nx.erdos_renyi_graph(n, pVal)
                adjacency_matrix = nx.to_numpy_array(graph)
                with open(sample_file, 'wb') as f:
                    pickle.dump(adjacency_matrix, f)

            adjacency_matrix_list.append(adjacency_matrix)

        adjacency_matrix_dict[n] = adjacency_matrix_list

    return adjacency_matrix_dict

# Configuration variables
nMin = 3
nMax = 100
nList = list(range(nMin, nMax, 1))  # List of n values
pVal = 0.8
samples = 100
t = 50  # Assuming 't' is a given parameter for the filename

# Base directory
base_dir = "Datasets/Benchmark-SimpleQWAK_ER/AdjacencyMatrices"

# Generate or load the adjacency matrix dictionary
adjacency_matrix_dict = create_or_load_adjacency_matrices(base_dir, nList, pVal, samples)


In [8]:
import os
import networkx as nx
import pickle
import numpy as np
from tqdm import tqdm

def create_or_load_adjacency_matrices(base_dir, n_values, pVal, samples):
    """
    Checks if adjacency matrix files exist for each size `n` and each sample. If not, generates and saves them.

    Parameters:
        base_dir (str): The base directory to save/load the adjacency matrices.
        n_values (list of int): A list of n values (graph sizes).
        pVal (float): The probability for edge creation in the Erdos-Renyi model.
        samples (int): The number of adjacency matrices to generate for each n value.

    Returns:
        dict: A dictionary where keys are `n` values and values are lists of adjacency matrices.
    """
    adjacency_matrix_dict = {}

    for n in tqdm(n_values, desc="Processing graph sizes"):
        n_dir = os.path.join(base_dir, f"N{n}")  # Directory for this n value
        os.makedirs(n_dir, exist_ok=True)

        adjacency_matrix_list = []
        for sample in tqdm(range(samples), desc=f"Samples for N={n}", leave=False):
            sample_file = os.path.join(n_dir, f"AdjMatrix_N{n}_P{pVal}_Sample{sample}.pkl")

            if os.path.exists(sample_file):
                # Load existing adjacency matrix
                print('File Exists!')
                with open(sample_file, 'rb') as f:
                    adjacency_matrix = pickle.load(f)
            else:
                # Generate and save a new adjacency matrix
                graph = nx.erdos_renyi_graph(n, pVal)
                adjacency_matrix = nx.to_numpy_array(graph)
                with open(sample_file, 'wb') as f:
                    pickle.dump(adjacency_matrix, f)

            adjacency_matrix_list.append(adjacency_matrix)

        adjacency_matrix_dict[n] = adjacency_matrix_list

    return adjacency_matrix_dict

# Configuration variables
nMin = 3
nMax = 100
nList = list(range(nMin, nMax, 1))  # List of n values
pVal = 0.8
samples = 100
t = 50  # Assuming 't' is a given parameter for the filename

# Base directory
base_dir = "Datasets/Benchmark-SimpleQWAK_ER/AdjacencyMatrices"

# Generate or load the adjacency matrix dictionary
adjacency_matrix_dict = create_or_load_adjacency_matrices(base_dir, nList, pVal, samples)


ples for N=3:   0%|        | 0/100 [00:00<?, ?it/s]
                                                   

File Exists!



ples for N=4:   0%|        | 0/100 [00:00<?, ?it/s]
                                                   

File Exists!



ples for N=5:   0%|        | 0/100 [00:00<?, ?it/s]
Processing graph sizes:   3%| | 3/97 [00:00<00:03, 27.

...


ples for N=97:  22%|▏| 22/100 [00:00<00:00, 97.73it
ples for N=97:  32%|▎| 32/100 [00:00<00:00, 97.18it
ples for N=97:  42%|▍| 42/100 [00:00<00:00, 97.99it
ples for N=97:  52%|▌| 52/100 [00:00<00:00, 97.11it
ples for N=97:  62%|▌| 62/100 [00:00<00:00, 97.07it
ples for N=97:  72%|▋| 72/100 [00:00<00:00, 97.27it
ples for N=97:  82%|▊| 82/100 [00:00<00:00, 96.46it
ples for N=97:  92%|▉| 92/100 [00:00<00:00, 96.59it
Processing graph sizes:  98%|▉| 95/97 [00:39<00:02,  1
ples for N=98:   0%|       | 0/100 [00:00<?, ?it/s]
ples for N=98:  11%| | 11/100 [00:00<00:00, 105.04i

File Exists!



ples for N=98:  22%|▏| 22/100 [00:00<00:00, 96.04it
ples for N=98:  32%|▎| 32/100 [00:00<00:00, 95.15it
ples for N=98:  43%|▍| 43/100 [00:00<00:00, 97.67it
ples for N=98:  53%|▌| 53/100 [00:00<00:00, 98.30it
ples for N=98:  64%|▋| 64/100 [00:00<00:00, 99.03it
ples for N=98:  74%|▋| 74/100 [00:00<00:00, 98.09it
ples for N=98:  84%|▊| 84/100 [00:00<00:00, 97.55it
ples for N=98:  94%|▉| 94/100 [00:00<00:00, 94.90it
Processing graph sizes:  99%|▉| 96/97 [00:40<00:01,  1
ples for N=99:   0%|       | 0/100 [00:00<?, ?it/s]
ples for N=99:  10%| | 10/100 [00:00<00:01, 89.44it

File Exists!



ples for N=99:  20%|▏| 20/100 [00:00<00:00, 92.23it
ples for N=99:  30%|▎| 30/100 [00:00<00:00, 91.66it
ples for N=99:  40%|▍| 40/100 [00:00<00:00, 91.17it
ples for N=99:  50%|▌| 50/100 [00:00<00:00, 91.30it
ples for N=99:  60%|▌| 60/100 [00:00<00:00, 92.09it
ples for N=99:  70%|▋| 70/100 [00:00<00:00, 90.44it
ples for N=99:  80%|▊| 80/100 [00:00<00:00, 91.02it
ples for N=99:  90%|▉| 90/100 [00:00<00:00, 90.51it
ples for N=99: 100%|█| 100/100 [00:01<00:00, 89.77i
Processing graph sizes: 100%|█| 97/97 [00:41<00:00,  2


In [ ]:
for q, qcp in zip(qw,qw_cupy):
    # Compare the two arrays using np.allclose
    are_close = np.allclose(q.getProbVec(), qcp.getProbVec(), atol=1e-5)

    # Print the result of the comparison
    print(f"Are the two arrays approximately equal? {are_close}")

In [ ]:
plt.plot(nList,qwak_times,label='QWAK CPU_NumPy')
plt.plot(nList,qwak_times_cupy,label='QWAK GPU_CuPy')
plt.legend()
plt.show()